In [1]:
import json
import pandas as pd
import numpy as np
import os

import re

from sqlalchemy import create_engine
import psycopg2

# from config import db_password

import time

In [2]:
# 1. Add the clean movie function that takes in the argument, "movie".
# Added from lessons8.3.6 clean movies combine the earlier function with the new
#merge columns function

def clean_movie(movie):
    movie = dict(movie) #create a non-destructive copy
    alt_titles = {}
    # combine alternate titles into one list
    for key in ['Also known as','Arabic','Cantonese','Chinese','French',
                'Hangul','Hebrew','Hepburn','Japanese','Literally',
                'Mandarin','McCune-Reischauer','Original title','Polish',
                'Revised Romanization','Romanized','Russian',
                'Simplified','Traditional','Yiddish']:
        if key in movie:
            alt_titles[key] = movie[key]
            movie.pop(key)
    if len(alt_titles) > 0:
        movie['alt_titles'] = alt_titles

    # merge column names
    def change_column_name(old_name, new_name):
        if old_name in movie:
            movie[new_name] = movie.pop(old_name)
    change_column_name('Adaptation by', 'Writer(s)')
    change_column_name('Country of origin', 'Country')
    change_column_name('Directed by', 'Director')
    change_column_name('Distributed by', 'Distributor')
    change_column_name('Edited by', 'Editor(s)')
    change_column_name('Length', 'Running time')
    change_column_name('Original release', 'Release date')
    change_column_name('Music by', 'Composer(s)')
    change_column_name('Produced by', 'Producer(s)')
    change_column_name('Producer', 'Producer(s)')
    change_column_name('Productioncompanies ', 'Production company(s)')
    change_column_name('Productioncompany ', 'Production company(s)')
    change_column_name('Released', 'Release Date')
    change_column_name('Release Date', 'Release date')
    change_column_name('Screen story by', 'Writer(s)')
    change_column_name('Screenplay by', 'Writer(s)')
    change_column_name('Story by', 'Writer(s)')
    change_column_name('Theme music composer', 'Composer(s)')
    change_column_name('Written by', 'Writer(s)')

    return movie

In [3]:
#delete later: create file path for testing

# 6 Create the path to your file directory and variables for the three files. 
file_dir = os.path.join("../Movies-ETL")
# Wikipedia data
wiki_file = f'{file_dir}/wikipedia_movies.json'
# Kaggle metadata
kaggle_file = f'{file_dir}/movies_metadata.csv'
# MovieLens rating data.
ratings_file = f'{file_dir}/ratings.csv'

In [177]:
#step 2 testing block

def ETL():
    
    # 2. Read in the kaggle metadata and MovieLens ratings CSV files as Pandas DataFrames.
    kaggle_file = pd.read_csv('movies_metadata.csv', low_memory = False)
    ratings_file = pd.read_csv('ratings.csv', low_memory = False)

    # 3. Open the read the Wikipedia data JSON file.
    with open(f'{file_dir}/wikipedia_movies.json', mode='r') as file:
        wiki_movies = json.load(file)
    
     # 3. Write a list comprehension to filter out TV shows.
    wiki_file = [movie for movie in wiki_movies
              if('Director' in movie or 'Directed by' in movie)
              and 'imdb_link' in movie
              and "no. of episodes" not in movie]
     # 4. Write a list comprehension to iterate through the cleaned wiki movies list
    # and call the clean_movie function on each movie.
    clean_movies = [clean_movie(movie) for movie in wiki_file]
    
    # 5. Read in the cleaned movies list from Step 4 as a DataFrame.
    wiki_movies_df = pd.DataFrame(clean_movies)
    sorted(clean_wiki_df.columns.tolist())
    
    # 6. Write a try-except block to catch errors while extracting the IMDb ID using a regular expression string and
    #  dropping any imdb_id duplicates. If there is an error, capture and print the exception.
    try:
        wiki_movies_df['imdb_id'] = clean_wiki_df['imdb_link'].str.extract(r'(tt\d{7})')
        wiki_movies_df.drop_duplicates(subset='imdb_id', inplace=True)
        
    except(IndexError):
        print('Index Error .... skipping')
        
     #  7. Write a list comprehension to keep the columns that don't have null values 
    #from the wiki_movies_df DataFrame.
    keep_columns = [column for column in wiki_movies_df.columns if wiki_movies_df[column].isnull().sum() 
                   <len(clean_wiki_df) * 0.9]
    wiki_movies_df = wiki_movies_df[keep_columns]
    
    # 8. Create a variable that will hold the non-null values from the “Box office” column.
    box_office = wiki_movies_df['Box office'].dropna()
    
    # 9. Convert the box office data created in Step 8 to string values using the lambda and 
    #join functions. Module 8.3.8
    box_office.map(lambda x: type(x) != str)
    box_office = box_office.apply(lambda x: ' '.join(x) if type(x) == list else x)
                               
    # 10. Write a regular expression to match the six elements of "form_one" of the box 
    #office data. 8.3.10
    form_one = r'\$\s*\d+\.?\d*\s*[mb]illi?on'                             
   # 11. Write a regular expression to match the three elements of "form_two" of the 
    #box office data.
    form_two = r'\$\s*\d{1,3}(?:[,\.]\d{3})+(?!\s[mb]illion)'

    # 12. Add the parse_dollars function. #8.3.10
    def parse_dollars(s):
        if type(s) != str:
            return np.nan
        if re.match(r'\$\s*\d+\.?\d*\s*milli?on', s, flags=re.IGNORECASE):
            s = re.sub('\$|\s|[a-zA-Z]','', s)
            value = float(s) * 10**6
            return value
        elif re.match(r'\$\s*\d+\.?\d*\s*billi?on', s, flags=re.IGNORECASE):
            s = re.sub('\$|\s|[a-zA-Z]','', s)
            value = float(s) * 10**9
            return value
        elif re.match(r'\$\s*\d{1,3}(?:[,\.]\d{3})+(?!\s[mb]illion)', s, flags=re.IGNORECASE):
            s = re.sub('\$|,','', s)
            value = float(s)
            return value
        else:
            return np.nan
    # 13. Clean the box office column in the wiki_movies_df DataFrame.
    wiki_movies_df['box_office'] = box_office.str.extract(f'({form_one}|{form_two})', 
                                    flags=re.IGNORECASE)[0].apply(parse_dollars)
    # 14. Clean the budget column in the wiki_movies_df DataFrame.
    

    # 15. Clean the release date column in the wiki_movies_df DataFrame.
    

    # 16. Clean the running time column in the wiki_movies_df DataFrame.
    
    # Return three variables. The first is the wiki_movies_df DataFrame
    
    return wiki_movies_df, kaggle_file, ratings_file

In [178]:
wiki_movies_df, kaggle_file, ratings_file = ETL()

In [179]:
len(wiki_movies_df)

7038

In [180]:
box_office

0          $21.4 million
1           $2.7 million
2            $57,718,089
3             $7,331,647
4       $6,939,946 (USA)
              ...       
7074       $19.4 million
7075       $41.9 million
7076       $76.1 million
7077       $38.4 million
7078        $5.5 million
Name: Box office, Length: 5485, dtype: object

In [127]:
type(box_office)

#words_df[words_df['text'].map(contains_x)]

pandas.core.series.Series

In [129]:
box_office.map(lambda x: type(x) !=str)
box_office = box_office.apply(lambda x: ' '.join(x) if type(x) == list else x)

In [130]:
box_office

0          $21.4 million
1           $2.7 million
2            $57,718,089
3             $7,331,647
4       $6,939,946 (USA)
              ...       
7074       $19.4 million
7075       $41.9 million
7076       $76.1 million
7077       $38.4 million
7078        $5.5 million
Name: Box office, Length: 5485, dtype: object

In [125]:

box_office = box_office.map(lambda x: type(x) !=str).apply (' '.join(x) 
                                                            if type(x) == list else x)

NameError: name 'x' is not defined

In [115]:
box_office

0       False
1       False
2       False
3       False
4       False
        ...  
7074    False
7075    False
7076    False
7077    False
7078    False
Name: Box office, Length: 5485, dtype: bool

In [166]:
box_office = clean_wiki_df['Box office'].dropna

In [172]:
len(x_office)

5485

In [175]:
 def is_not_a_string(x):
        return type(x) != str
        

In [178]:
box_office[x_office.map(lambda x: type(x) != str)]

34                           [US$, 4,212,828]
54      [$6,698,361 (, United States, ), [2]]
75                    [$6,488,144, (US), [1]]
127                [US$1,531,489, (domestic)]
131                          [US$, 4,803,039]
                        ...                  
6984               [$99.6, million, [4], [5]]
6998                   [$365.6, million, [1]]
6999                         [$53.8, million]
7019                     [$435, million, [7]]
7052                   [$529.3, million, [4]]
Name: Box office, Length: 135, dtype: object

In [134]:
len(clean_wiki_df)

7037

In [177]:
box_office

34                           [US$, 4,212,828]
54      [$6,698,361 (, United States, ), [2]]
75                    [$6,488,144, (US), [1]]
127                [US$1,531,489, (domestic)]
131                          [US$, 4,803,039]
                        ...                  
6984               [$99.6, million, [4], [5]]
6998                   [$365.6, million, [1]]
6999                         [$53.8, million]
7019                     [$435, million, [7]]
7052                   [$529.3, million, [4]]
Name: Box office, Length: 135, dtype: object

In [136]:
box_office = box_office[box_office.map(is_not_a_string)]

AttributeError: 'function' object has no attribute 'map'

In [ ]:
clean_wiki_df.info()

In [31]:


    
    # 14. Clean the budget column in the wiki_movies_df DataFrame.
    

    # 15. Clean the release date column in the wiki_movies_df DataFrame.
    

    # 16. Clean the running time column in the wiki_movies_df DataFrame.
    
    # Return three variables. The first is the wiki_movies_df DataFrame
    
    return wiki_file
                 #clean_wiki_df #kaggle_file, ratings_file

In [4]:
# 17. Create the path to your file directory and variables for the three files.
# 6 Create the path to your file directory and variables for the three files. 
file_dir = os.path.join("../Movies-ETL")
# Wikipedia data
wiki_file = f'{file_dir}/wikipedia_movies.json'
# Kaggle metadata
kaggle_file = f'{file_dir}/movies_metadata.csv'
# MovieLens rating data.
ratings_file = f'{file_dir}/ratings.csv'

In [5]:
# 18. Set the three variables equal to the function created in D1.
##changed func name to match with mine
wiki_file, kaggle_file, ratings_file = ETL()

In [6]:
# 19. Set the wiki_movies_df equal to the wiki_file variable. 
wiki_movies_df = wiki_file

In [7]:
# 20. Check that the wiki_movies_df DataFrame looks like this. 
wiki_movies_df.head()

In [8]:
# 21. Check that wiki_movies_df DataFrame columns are correct. 
wiki_movies_df.columns.to_list()